<p style="text-align:center">
</p>

<h1 align=center><font size = 5>SpaceX Falcon 9 First Stage Landing Prediction</font></h1>


## Introduction

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a database
3.  Execute SQL queries to answer the questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 67.4 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=39e85659d8190ea12a996ab4458d62c3fa790271e7ca2849bef9356e303cf935
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


**Note:This below code is added to remove blank rows from table**


In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]


##### Display the names of the unique launch sites  in the space mission


In [9]:
%sql select Distinct(Launch_Site) from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



#####  Display 5 records where launch sites begin with the string 'CCA' 


In [10]:
%sql select * from SPACEXTABLE where Launch_Site Like 'CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
6/4/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
12/8/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
10/8/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
3/1/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [15]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTABLE where Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596



##### Display average payload mass carried by booster version F9 v1.1


In [18]:
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTABLE where Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2928.4


##### List the date when the first succesful landing outcome in ground pad was acheived.


In [42]:
%sql select Date from SPACEXTABLE where Landing_Outcome='Success (ground pad)' limit 1

 * sqlite:///my_data1.db
Done.


Date
22/12/2015




##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [45]:
%sql select Booster_Version,Landing_Outcome, PAYLOAD_MASS__KG_ from SPACEXTABLE where Landing_Outcome='Success (drone ship)' and PAYLOAD_MASS__KG_ between 4000 and 6000

 * sqlite:///my_data1.db
Done.


Booster_Version,Landing_Outcome,PAYLOAD_MASS__KG_
F9 FT B1022,Success (drone ship),4696
F9 FT B1026,Success (drone ship),4600
F9 FT B1021.2,Success (drone ship),5300
F9 FT B1031.2,Success (drone ship),5200







##### List the total number of successful and failure mission outcomes


In [62]:
%sql select count(Mission_Outcome) as Total,(select count(Mission_Outcome) from SPACEXTABLE where Mission_Outcome='Failure (in flight)')as Failure, count(Mission_Outcome)-(select count(Mission_Outcome) from SPACEXTABLE where Mission_Outcome='Failure (in flight)') as Success from SPACEXTABLE 

 * sqlite:///my_data1.db
Done.


Total,Failure,Success
101,1,100


In [58]:
%sql select distinct(Mission_Outcome) from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Mission_Outcome
Success
Failure (in flight)
Success (payload status unclear)
Success


In [63]:
%sql select * from SPACEXTABLE limit 1

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
6/4/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)





##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [65]:
%sql select Booster_Version,PAYLOAD_MASS__KG_ from SPACEXTABLE where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600



##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So  need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [122]:
%sql select Date,substr(Date,3,3) as month,Landing_Outcome,Booster_Version,Launch_Site from SPACEXTABLE where substr(Date,-4,4)='2015' and Landing_Outcome='Failure (drone ship)'

 * sqlite:///my_data1.db
Done.


Date,month,Landing_Outcome,Booster_Version,Launch_Site
1/10/2015,10/,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
14/04/2015,/04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40



##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [148]:
%sql select Landing_Outcome,count(Landing_Outcome) as count from SPACEXTABLE where substr(Date,-4,4) between '2010' and '2017' Group by Landing_Outcome Having Landing_Outcome='Failure (drone ship)' or Landing_Outcome='Success (ground pad)' order by count Desc 

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count
Success (ground pad),8
Failure (drone ship),5


In [159]:
%sql select * from SPACEXTABLE where (substr(Date,-4,4) between '2010' and '2017') and (Landing_Outcome='Failure (drone ship)' or Landing_Outcome='Success (ground pad)')

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
1/10/2015,9:47:00,F9 v1.1 B1012,CCAFS LC-40,SpaceX CRS-5,2395,LEO (ISS),NASA (CRS),Success,Failure (drone ship)
14/04/2015,20:10:00,F9 v1.1 B1015,CCAFS LC-40,SpaceX CRS-6,1898,LEO (ISS),NASA (CRS),Success,Failure (drone ship)
22/12/2015,1:29:00,F9 FT B1019,CCAFS LC-40,OG2 Mission 2 11 Orbcomm-OG2 satellites,2034,LEO,Orbcomm,Success,Success (ground pad)
17/01/2016,18:42:00,F9 v1.1 B1017,VAFB SLC-4E,Jason-3,553,LEO,NASA (LSP) NOAA CNES,Success,Failure (drone ship)
3/4/2016,23:35:00,F9 FT B1020,CCAFS LC-40,SES-9,5271,GTO,SES,Success,Failure (drone ship)
15/06/2016,14:29:00,F9 FT B1024,CCAFS LC-40,ABS-2A Eutelsat 117 West B,3600,GTO,ABS Eutelsat,Success,Failure (drone ship)
18/07/2016,4:45:00,F9 FT B1025.1,CCAFS LC-40,SpaceX CRS-9,2257,LEO (ISS),NASA (CRS),Success,Success (ground pad)
19/02/2017,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
5/1/2017,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
6/3/2017,21:07:00,F9 FT B1035.1,KSC LC-39A,SpaceX CRS-11,2708,LEO (ISS),NASA (CRS),Success,Success (ground pad)
